In [2]:
#from crypt import methods
from distutils.command.config import config
from flask import Flask, flash, redirect, url_for, render_template, request, Response
import pandas as pd
from bs4 import BeautifulSoup
import requests, json, ast
import os, io, csv, sys, pickle, time
from collections import Counter
from werkzeug.utils import secure_filename

In [12]:
from googlesearch import search
# to search
query = 'six of crows' + ' goodreads'
searched_url = next(search(query, num=1, stop=0,  pause=0))

In [13]:
searched_url

'https://www.goodreads.com/book/show/23437156-six-of-crows'

In [10]:
book_dict = pd.read_pickle(r"C:\Users\Lenovo\Flask apps\flask-app-character-net\uploaded_files\book_dict.pkl")
i=0
all_sentences = []
list_corpus = book_dict['book_content']
list_corpus = [sent for sent in list_corpus.split('\n') if sent !='']
list_corpus[50*i:50*(i+1)]

['T hey stayed up planning well past midnight. Kaz was wary of the changes to the plan as well as the prospect of managing Nina’s pack of Grisha. But though he gave no indication to the others, there were elements of this new course that appealed to him. It was possible that Van Eck would piece together what the Shu were doing and go after the city’s remaining Grisha himself. They were a weapon Kaz didn’t want to see in the mercher’s arsenal.',
 'But they couldn’t let this little rescue slow them down. With so many opponents and the stadwatch involved, they couldn’t afford it. Given enough time, the Shu would stop worrying about those dry-docked warships and the Council of Tides, and find their way to Black Veil. Kaz wanted Kuwei out of the city and removed from play as soon as possible.',
 'At last, they put their lists and sketches aside. The wreckage of their makeshift meal was cleared from the table to avoid attracting the rats of Black Veil, and the lanterns were doused.',
 'The o

In [16]:

resp = requests.get(searched_url)

soup = BeautifulSoup(resp.content, 'html.parser')

for x in soup.find_all("a", { "class" : "actionLinkLite bookPageGenreLink" }):

    print(x.get_text())

Fantasy
Young Adult
Fiction
Young Adult
Young Adult Fantasy
Adventure
LGBT
Fantasy
Magic
Romance
Fantasy
High Fantasy
Audiobook


In [ ]:
from googlesearch import search
from distutils.command.config import config
from flask import Flask, flash, redirect, url_for, render_template, request, Response
import pandas as pd
from bs4 import BeautifulSoup
import requests, json, ast
import os, io, csv, sys, pickle, time
from collections import Counter
from werkzeug.utils import secure_filename


# to search
query = 'six of crows' + ' goodreads'
searched_url = next(search(query, num=1, stop=0,  pause=0))


resp = requests.get(searched_url)
soup = BeautifulSoup(resp.content, 'html.parser')

for x in soup.find_all("a", { "class" : "actionLinkLite bookPageGenreLink" }):

    print(x.get_text())

In [18]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [17]:
!git lfs install
!git clone https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

Updated git hooks.
Git LFS initialized.


Cloning into 'distilbert-base-uncased-finetuned-sst-2-english'...
fatal: unable to access 'https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/': OpenSSL SSL_connect: Connection was reset in connection to huggingface.co:443 


In [19]:
!pip install -r /content/character-network/requirements_sentiment_analysis.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/character-network/requirements_sentiment_analysis.txt'


In [3]:
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\book_content.pkl', 'rb') as f:
    book_content = f.read()
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\sentiment_lables.pkl', 'rb') as f:
    sentiment_lables = f.read()
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\encoded_sentiment_labels.pkl', 'rb') as f:
    encoded_sentiment_labels = f.read()
with open(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\emotions_count.pkl', 'r') as f:
    emotions_count = f.read()

In [6]:
pd.read_pickle(r'C:\Users\Lenovo\flask-app-character-net\first_book_props\emotions_count.pkl')

{'POSITIVE': 2530, 'NEGATIVE': 3896, 'NEUTRAL': 258}

In [4]:
emotions_count

'€\x03}q\x00(X\x08\x00\x00\x00POSITIVEq\x01Mâ\tX\x08\x00\x00\x00NEGATIVEq\x02M8\x0fX\x07\x00\x00\x00NEUTRALq\x03M\x02\x01u.'

In [ ]:
# sentiment analysis on sentences
# we use gpu for this task

def senti_analysis_transformers(sentences, plot=True):


    """
    given the fact that we have GPU on colab
    We empty the cache and initialize our sentiment analysis
    """
    print('Defining the sentiment of each sentence using a semantic model...')
    torch.cuda.empty_cache()

    sentiments_lists = []

    # we use small batches to prevent crashing
    for i in range(0, len(sentences), 50):
        if i%1000==0 : print('sentence', colored(f"{i}", 'blue'))
        sentiments_lists.append(classifier(sentences[i: i+50]))
    
    # list of list -> list
    all_sentences_sentiment = list(chain.from_iterable(sentiments_lists))

    labels, encoded_labels = decide_for_sentiment_label(all_sentences_sentiment)

    
    # count the labels
    
    emotions_count = dict(Counter(labels))
    print(colored('Emotions dominance: \n', 'blue'), emotions_count)
    
    if plot:plot_emotions(emotions_count)
    
    return labels, encoded_labels, emotions_count

sentiment_lables, encoded_sentiment_labels, emotions_count = senti_analysis_transformers(book_sents_cleaned, plot=False)


In [11]:

from afinn import Afinn

afn = Afinn()

afn.score('we are kind to good people')

5.0

In [ ]:

    sent_dict = {}
    afinn = Afinn()
    for sent in sentence_list:
        sent_dict[sent] = afinn.score(sent)

    #sentiment_score = [x: afinn.score(x) for x in sentence_list]
    #align_rate = np.sum(sentiment_score)/len(np.nonzero(sentiment_score)[0]) * -2
    sent_df = pd.DataFrame({'sentiment': sent_dict.values(),
 'sentence': sent_dict.keys()})
    return sent_dict, sent_df

sent_dict, sent_df = calculate_align_rate(book_1_sents)

In [21]:
sentiment = ['positive' if score > 0 
            else 'negative' if score < 0 
            else 'neutral' 
            for score in scores]

NameError: name 'scores' is not defined

In [22]:

from afinn import Afinn

afn = Afinn()
senti_analysis_Afinn(['this is ok', 'fucked up.'])

(['NEUTRAL', 'NEGATIVE'], [0.0, -0.8], {'NEUTRAL': 1, 'NEGATIVE': 1})

In [17]:
def senti_analysis_Afinn(sentence_list:list):
    '''
    Function to calculate the align_rate of the whole novel
    :param sentence_list: the list of sentence of the whole novel.
    :return: the align rate of the novel.
    '''
    afn = Afinn()

    # encoded_labels = sentiments_lists in transformers
    encoded_labels = []

    # the score is divided to be compatible with the transformers approach
    # |max or min Afinn| = 5
    for sent in sentence_list:
        encoded_labels.append(afn.score(sent)/5)
    

    labels = sentiment = [
        'POSITIVE' if score > 0 
        else 'NEGATIVE' if score < 0 
        else 'NEUTRAL' 
        for score in encoded_labels
        ]

    
    # count the labels
    emotions_count = dict(Counter(labels))
    #print(colored('Emotions dominance: \n', 'blue'), emotions_count)
    
    #if plot:plot_emotions(emotions_count)
    
    return labels, encoded_labels, emotions_count

In [6]:
import pandas as pd
pd.DataFrame({'Top':i, 'Known as':k, 'Num. of Appearances':v} for i, (k,v) in enumerate({'sdfsd':5, 'sdfsg': 10}.items()))

,top,this,that
0,0,sdfsd,5
1,1,sdfsg,10


In [ ]:
def decide_for_transformer_sentiment_label(self, senti_dicts:dict) ->list:
    """
    takes in dictionaries of sentiment analysis
    in this format: {'label': '...', 'score':...} 
    (label: 'POSITIVE'/'NEGATIVE', 'score': 0<s<1)
    outputs a list representing the emotion of each sentence
    """
    labels = []

    for senti_dict in senti_dicts:

        # senti_dict['score'] = how certain the model is about an emotion
        if float(senti_dict['score'])< 0.7:
            labels.append('NEUTRAL')

        else: labels.append(senti_dict['label'])

    # encode the labels
    emotions_label = {'POSITIVE': 1, 'NEUTRAL':0, 'NEGATIVE':-1}

    # encode the three types
    encoded_lables = [emotions_label[em] for em in labels]
    
    

    return labels, encoded_lables




# sentiment analysis on sentences
# we use gpu for this task


# new phase

In [1]:

import codecs
import os
import spacy
import json
import pandas as pd
import regex as re
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
from afinn import Afinn
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
import spacy
from termcolor import colored
import time
from itertools import chain 
import string

from itertools import chain

import os
from tqdm import tqdm 
import tqdm.notebook as tq
pd.set_option('display.max_colwidth', None)
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import io

In [2]:

sys.path.append(r'C:\Users\Lenovo\character-network')

In [3]:
from character_net_src import Book_analyzer

analyzer = Book_analyzer()


book_content_cleaned = analyzer.clean_sentences("""They’d wanted him to die without honor, murdered in an infirmary bed.
Despite the fact that she’d been as good as starving herself, the vent was still a tight fit.
She’d woken sometime after the fight on Vellgeluk, with no sense of how long she’d been unconscious and no idea where she was.
She’d managed to nudge her blindfold down by scraping her face against the wall.
She’d leashed it by controlling her breath, in through the nose, out through the mouth, letting her mind turn to prayer as her Saints gathered around her.
We meet fear , he’d said.
He was just like the boys she’d grown up with, a head full of nonsense and a mouth full of easy charm.
She’d fight with everything she had to free Inej even if she was still in the grips of parem .
Van Eck had sent Bajan to her every day, and he’d been nothing but amiable and pleasant even as he’d prodded her for the locations of Kaz’s safe houses.
Or maybe he thought she’d be more vulnerable to a Suli boy than a wily merch.
By the time they’d made it to the main street, he had a respectable little bunch.
Then he felt a little like the first time he’d tried brandy and ended up spewing his dinner all over his own shoes.
“You’d be surprised,” said Jesper.
Wylan clutched his side, wishing he’d shoved Jesper out of the wagon after all and jumped right down with him.
But they’d come all this way.""")

book_sentences = analyzer.spacy_detect_sentences(book_content_cleaned)

finalized_sents = analyzer.clean_sentences(book_sentences, chapter_regex = 'No chapter')


In [4]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
len(f" *{string.punctuation}\w *")

38

In [11]:
print(f" *{string.punctuation}\w *")

 *!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~\w *


In [7]:
re.findall(pattern=f" *[{string.punctuation}’”]\w *",
 string="They’d wanted him to die without honor, murdered in an infirmary bed.")

['’d ']

In [4]:
path_1 = r"C:\Users\Lenovo\character-network\novels\Harry Potter 1.txt"
path_2 = r'C:\Users\Lenovo\Desktop\croocked.txt'

from charset_normalizer import from_path

results = from_path(path_2)

book_content = str(results.best())

In [5]:
# ner -----------------------------------------
n=10
sorted_flatten_names_dict = analyzer.flatten_pop_names(list_sents=finalized_sents)
df = pd.DataFrame({'Top':i, 'Known as':k, 'Num. of Appearances':v} for i, (k,v) in enumerate(sorted_flatten_names_dict.items()))
top_n_df = df.iloc[:n, :]

doc object text (doc.text) =  They’d wanted him to die without honor, murdered in an infirmary bed.
name detected by spacy (ent.text) =  ’d
found!
doc object text (doc.text) =  She’d managed to nudge her blindfold down by scraping her face against the wall.
name detected by spacy (ent.text) =  ’d
found!
doc object text (doc.text) =  She’d leashed it by controlling her breath, in through the nose, out through the mouth, letting her mind turn to prayer as her Saints gathered around her.
name detected by spacy (ent.text) =  ’d
found!
doc object text (doc.text) =  She’d fight with everything she had to free Inej even if she was still in the grips of parem .
name detected by spacy (ent.text) =  ’d
found!
doc object text (doc.text) =  Van Eck had sent Bajan to her every day, and he’d been nothing but amiable and pleasant even as he’d prodded her for the locations of Kaz’s safe houses.
name detected by spacy (ent.text) =  Van Eck
doc object text (doc.text) =  Van Eck had sent Bajan to her eve

In [6]:
top_n_df

,Top,Known as,Num. of Appearances
0,0,Van Eck,1
1,1,Suli,1
2,2,Jesper,1
3,3,Wylan,1
